# 08 Merge VRS Realtime data with Trips data

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
import hashlib

In [2]:
# Set month
data_month_single = '7'
data_month_double = '07'
data_month_int = 7

In [3]:

# load routes data from notebook 4
routes = pd.read_csv('../Data/04_routes_merged.csv')

## Trips

Prepare trips and merge with routes for later merge with realtime data

In [4]:
# load trips data
trips = pd.DataFrame(pd.read_csv("../Data/01-000000-UTC_gtfs-scraper_trips-vrs.csv"))

In [5]:
# merge the trips and routes DataFrames based on the route_id column
trips = pd.merge(routes, trips, on='route_id')

In [6]:
trips

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,shape_id
0,100001,1,1,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26781,272-1-001-200.1.11:051300-24-2_C43BA7A0-464B-4...,Brück Mauspfad,0,514
1,100001,1,1,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26781,273-1-001-200.1.11:054300-24-2_7C70A3CB-C405-4...,Brück Mauspfad,0,514
2,100001,1,1,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26985,1446-1-001-200.1.11:051300-24-256_0BEF39DE-01E...,Brück Mauspfad,0,514
3,100001,1,1,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26985,1447-1-001-200.1.11:054300-24-256_A4EA3F13-647...,Brück Mauspfad,0,514
4,100001,1,1,0,Tram,KVB Kölner Verkehrs-Betriebe AG,49,130-1-001-200.1.11:150200-24-2_1FFFE10D-006B-4...,Brück Mauspfad,0,514
...,...,...,...,...,...,...,...,...,...,...,...
111950,19911791,5,S6,3,Bus,"DB DB - Regio AG, Region NRW",4489,7053-1206-005-572.7:122100-6-13_F816F8EF-E0B7-...,Langenfeld (S),0,4666
111951,19911791,5,S6,3,Bus,"DB DB - Regio AG, Region NRW",4489,7167-1206-005-572.7:125100-6-13_0BFA83D1-E6BA-...,Langenfeld (S),0,4666
111952,19911791,5,S6,3,Bus,"DB DB - Regio AG, Region NRW",4489,7059-1206-005-572.7:132100-6-13_15652C30-C531-...,Langenfeld (S),0,4666
111953,19911791,5,S6,3,Bus,"DB DB - Regio AG, Region NRW",4489,7171-1206-005-572.7:135100-6-13_BADC0F45-4BBF-...,Langenfeld (S),0,4666


In [7]:
# trips in the other data have three digits as max
trips['route_id'] = trips['route_id'].astype(str).str[-3:]

In [8]:
# some ids have only two digits
trips['route_id'] = trips['route_id'].astype(str).apply(lambda x: x[-2:] if x[0] == '0' else x)

In [9]:
# read realtime data
realtime = pd.DataFrame(pd.read_csv("../Data/06_VRS_with_timestamp.csv"))


In [10]:
realtime

,trip_id,start_date,schedule_relationship,route_id,stop_id,stop_arrival_time,stop_departure_time,vrs_timestamp
0,6870009-687-006-687.2.24:253500-42-1_FA880471-...,20230630,0,687,687,0,2023-07-01 01:35:00,308
1,6870009-687-006-687.2.24:253500-42-1_FA880471-...,20230630,0,687,692,2023-07-01 01:40:30,2023-07-01 01:40:45,308
2,6870009-687-006-687.2.24:253500-42-1_FA880471-...,20230630,0,687,691,2023-07-01 01:41:45,2023-07-01 01:42:00,308
3,6870009-687-006-687.2.24:253500-42-1_FA880471-...,20230630,0,687,690,2023-07-01 01:42:45,2023-07-01 01:43:00,308
4,6870009-687-006-687.2.24:253500-42-1_FA880471-...,20230630,0,687,698,2023-07-01 01:44:00,2023-07-01 01:44:15,308
...,...,...,...,...,...,...,...,...
75056347,1918-16-001-343.1.12:243300-37-260_608FDFB6-AD...,20230731,0,16,694,2023-08-01 01:31:00,2023-08-01 01:31:00,234305
75056348,1918-16-001-343.1.12:243300-37-260_608FDFB6-AD...,20230731,0,16,693,2023-08-01 01:33:00,2023-08-01 01:33:00,234305
75056349,1918-16-001-343.1.12:243300-37-260_608FDFB6-AD...,20230731,0,16,43,2023-08-01 01:35:00,2023-08-01 01:35:00,234305
75056350,1918-16-001-343.1.12:243300-37-260_608FDFB6-AD...,20230731,0,16,688,2023-08-01 01:36:00,2023-08-01 01:36:00,234305


In [11]:
realtime['route_id'].unique()

array([687, 684,  66, 640, 682, 685, 688, 686, 551, 855, 856, 857, 845,
        55, 537,  16,  18,  60, 550,  65, 611, 610, 609, 612, 605, 607,
       604, 602, 608, 843, 603, 638, 606, 601, 613, 614, 600, 639, 631,
       636, 633, 800, 516, 817, 884, 632, 630, 635,  63, 540, 637, 689,
       683, 690, 681, 541, 842,  68, 634, 662])

In [12]:
realtime.drop_duplicates(inplace=True)

In [13]:
realtime

,trip_id,start_date,schedule_relationship,route_id,stop_id,stop_arrival_time,stop_departure_time,vrs_timestamp
0,6870009-687-006-687.2.24:253500-42-1_FA880471-...,20230630,0,687,687,0,2023-07-01 01:35:00,308
1,6870009-687-006-687.2.24:253500-42-1_FA880471-...,20230630,0,687,692,2023-07-01 01:40:30,2023-07-01 01:40:45,308
2,6870009-687-006-687.2.24:253500-42-1_FA880471-...,20230630,0,687,691,2023-07-01 01:41:45,2023-07-01 01:42:00,308
3,6870009-687-006-687.2.24:253500-42-1_FA880471-...,20230630,0,687,690,2023-07-01 01:42:45,2023-07-01 01:43:00,308
4,6870009-687-006-687.2.24:253500-42-1_FA880471-...,20230630,0,687,698,2023-07-01 01:44:00,2023-07-01 01:44:15,308
...,...,...,...,...,...,...,...,...
75056347,1918-16-001-343.1.12:243300-37-260_608FDFB6-AD...,20230731,0,16,694,2023-08-01 01:31:00,2023-08-01 01:31:00,234305
75056348,1918-16-001-343.1.12:243300-37-260_608FDFB6-AD...,20230731,0,16,693,2023-08-01 01:33:00,2023-08-01 01:33:00,234305
75056349,1918-16-001-343.1.12:243300-37-260_608FDFB6-AD...,20230731,0,16,43,2023-08-01 01:35:00,2023-08-01 01:35:00,234305
75056350,1918-16-001-343.1.12:243300-37-260_608FDFB6-AD...,20230731,0,16,688,2023-08-01 01:36:00,2023-08-01 01:36:00,234305


In [14]:
# filter for trips only being in realtime data
#trips_filtered_on_id = trips[trips['trip_id'].isin(realtime['trip_id'])]

In [15]:
# change route_id format to 
#trips_filtered_on_id['route_id'] = trips_filtered_on_id['route_id'].astype(str).str.slice(3, 6)

In [16]:
#trips_filtered_on_id['route_id'] = trips_filtered_on_id['route_id'].astype(int)

In [17]:
#merged_df = trips_filtered_on_id.merge(realtime, on=['trip_id', 'route_id'])
trips['route_id'] = trips['route_id'].astype(int)


In [18]:
merged_df = trips.merge(realtime, on=['trip_id', 'route_id'])

In [19]:
#export to csv
merged_df.to_csv('../Data/07_vrs_merged_with_trips_NEU.csv', index=False)

In [20]:
merged_df

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,shape_id,start_date,schedule_relationship,stop_id,stop_arrival_time,stop_departure_time,vrs_timestamp
0,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,0,3004,20230701,0,371,0,2023-07-01 05:17:00,24804
1,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,0,3004,20230701,0,161,2023-07-01 05:18:00,2023-07-01 05:18:00,24804
2,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,0,3004,20230701,0,701,2023-07-01 05:19:00,2023-07-01 05:19:00,24804
3,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,0,3004,20230701,0,700,2023-07-01 05:20:00,2023-07-01 05:20:00,24804
4,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,0,3004,20230701,0,699,2023-07-01 05:21:00,2023-07-01 05:21:00,24804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71378312,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,1,8598,20230731,0,5573,0,0,185304
71378313,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,1,8598,20230731,0,1333,0,0,185304
71378314,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,1,8598,20230731,0,1337,0,0,185304
71378315,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,1,8598,20230731,0,1335,0,0,185304


In [21]:
merged_df['route_id'].unique()

array([ 16,  18,  55,  60,  63,  65,  66,  68, 540, 600, 601, 602, 603,
       604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 630, 631,
       632, 633, 634, 635, 636, 637, 638, 639, 640, 662, 681, 682, 683,
       684, 685, 686, 687, 688, 689, 690, 516, 537, 541, 550, 551, 800,
       817, 842, 843, 845, 855, 856, 857, 884])